<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/03_learning/07_rules_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os

from fuzzy_systems.learning.mandani_learning import MamdaniLearning
from fuzzy_systems import MamdaniSystem


import numpy as np
import matplotlib.pyplot as plt

# Configurar matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['figure.dpi'] = 100


from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import pandas as pd


In [ ]:
# Load dataset
iris = load_iris()

X = iris.data # Shape (150, 4) - 4 features
y = iris.target
y_binary = (y != 0).astype(int)  # 0 = Setosa, 1 = Não-Setosa


feature_names = iris.feature_names 
class_names = iris.target_names

X = X[:,2:4]
feature_names = ['Petal Length (cm)', 'Petal Width (cm)']
class_names = ['setosa', 'versicolor', 'virginica']


#Visualize separability
fig, ax = plt.subplots(figsize=(10, 8))

colors = ['red', 'blue']
labels = ['Setosa', 'Non-Setosa']

for class_id in [0, 1]:
    mask = y_binary == class_id
    ax.scatter(X[mask, -2], X[mask, -1], 
              c=colors[class_id], label=labels[class_id],
              s=80, alpha=0.7, edgecolors='black', linewidth=1)

ax.set_xlabel(iris.feature_names[-2] + ' (cm)', fontsize=13)
ax.set_ylabel(iris.feature_names[-1] + ' (cm)', fontsize=13)
ax.set_title('Iris Dataset - 2 Features (Petal)', fontsize=15, weight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_binary, test_size=0.3, random_state=42, stratify=y_binary
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Normalize data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)


for i in range(X_train.shape[1]):
    print(f'Feature {i+1} - Train: [{X_train_norm[:, i].min():.2f}, {X_train_norm[:, i].max():.2f}] | '
          f'Validation: [{X_val_norm[:, i].min():.2f}, {X_val_norm[:, i].max():.2f}] | '
          f'Test: [{X_test_norm[:, i].min():.2f}, {X_test_norm[:, i].max():.2f}]')

In [ ]:
# ==================== 3. CRIAR SISTEMA FUZZY ====================

fis = MamdaniSystem()
num_features = X_train.shape[1]
num_input_mfs = 3
for idx in range(num_features):
    universe = (X_train_norm[:,idx].min(), X_train_norm[:,idx].max())
    fis.add_input(feature_names[idx], universe)
    input_centers = np.linspace(universe[0], universe[1], num_input_mfs)
    del_in = (input_centers[1]-input_centers[0])
    for i in range(num_input_mfs):
        a =  input_centers[i] - del_in
        b =  input_centers[i]
        c =  input_centers[i] + del_in
        fis.add_term(feature_names[idx], f'MF_{i+1}', 'triangular', (a, b, c))


fis.add_output('setosa', (0, 1))
num_output_mfs = 2
fis.add_term('setosa', 'No', 'triangular', (0, 0.0, 1.0))
fis.add_term('setosa', 'Yes', 'triangular', (0.0, 1.0, 1.0))
fis.info()

In [ ]:
learner = MamdaniLearning(fis, num_points=1000, verbose=True)

In [ ]:
learner.fit_rules(
            X_train_norm, y_train,
            optimizer='sa',
            optimizer_params={'temperature_init': 100.0, 'cooling_rate': 0.95, 'max_iterations': 5000,'temperature_min': 1e-6},
            initial_solution_method='random'
        )

y_pred_train = learner.predict(X_train_norm)
y_pred_test = learner.predict(X_test_norm)

rmse_train = learner.get_cost()
rmse_test = learner.score(X_test_norm, y_test)

In [ ]:
y_pred_test_classes = (y_pred_test >= 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred_test_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names[1:])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Iris Dataset (Non-Setosa vs Setosa)')
plt.show()

In [ ]:
rule_matrix = fis.plot_rule_matrix()

In [ ]:
history = learner.get_history()
plt.figure(figsize=(10, 6))
plt.plot(history['current_cost'], label='Training Cost', color='blue')
plt.plot(history['best_cost'], label='Best Cost', color='red')
plt.xlabel('Iteration')
plt.ylabel('Cost (RMSE)')
plt.title('Training Cost over Iterations')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df = fis.rules_to_dataframe()
df

In [ ]:
# Configurações dos otimizadores
optimizers_config = {
    'SA': {
        'optimizer': 'sa',
        'params': {
            'temperature_init': 100.0,
            'cooling_rate': 0.95,
            'max_iterations': 2000,
            'plateau_iterations': 500,
            'temperature_min': 1e-6
        }
    },
    'GA': {
        'optimizer': 'ga',
        'params': {
            'pop_size': 50,
            'max_gen': 100,
            'mutation_rate': 0.1,
            'crossover_rate': 0.8
        }
    },
    'PSO': {
        'optimizer': 'pso',
        'params': {
            'n_particles': 30,
            'n_iterations': 100,
            'w_max': 0.9,
            'w_min': 0.4
        }
    },
    'DE': {
        'optimizer': 'de',
        'params': {
            'pop_size': 30,
            'max_iter': 100,
            'F': 0.8,
            'CR': 0.7
        }
    }
}


In [ ]:
learner = MamdaniLearning(fis, num_points=1000, verbose=True)
optimizer ='GA'
learner.fit_rules(
            X_train_norm, y_train,
            optimizer=optimizers_config[optimizer]['optimizer'],
            optimizer_params=optimizers_config[optimizer]['params'],
            initial_solution_method='random'
        )